In [1]:
import math
import json
import os
import pandas as pd
import numpy as np

from pyecharts import options as opts
from pyecharts.charts import Graph, Page

In [2]:
file_path = r'C:\Users\yyrhw\Desktop\citation_datasets\citation_author_deprived.csv'

In [3]:
datasource_次数 = pd.read_csv(file_path)

In [4]:
# 此次可视化暂不涉及时间维度，因此将其drop
datasource_次数 = datasource_次数.drop('施引年份', axis=1)

In [5]:
# 将重复的作者名删去，以防误求和
datasource_次数 = datasource_次数.drop_duplicates(subset=['被引作者'], keep='first')

In [6]:
# 分组求和
datasource_次数 = datasource_次数.groupby('被引作者').sum().reset_index()

In [7]:
datasource_次数.head()

,被引作者,施引次数
0,"Antonoglou, Ioannis",165
1,"Baker, Lucas",76
2,"Barfett, Joseph",22
3,Binh Thai Pham,37
4,"Blau, Helen M.",118


In [8]:
# 该组数据只保留施引被引关系，施引年份和施引次数直接丢弃
datasource_引用 = pd.read_csv(file_path).drop(['施引年份', '施引次数'], axis=1)
datasource_引用.head(5)

,被引作者,施引作者
0,"Antonoglou, Ioannis","Montavon, Gregoire"
1,"Lillicrap, Timothy","Binder, Alexander"
2,"Schrittwieser, Julian","Mueller, Klaus-Robert"
3,"Graepel, Thore","Mueller, Klaus-Robert"
4,"Lillicrap, Timothy","Lapuschkin, Sebastian"


In [9]:
nodes = []
links = []
# 遍历两个DataFrame的行，构建关系图节点和连线
for index, Series in datasource_次数.iterrows():
    nodes.append(opts.GraphNode(name=Series[0], symbol_size=math.sqrt(Series[1])))
for index, Series in datasource_引用.iterrows():
    links.append(opts.GraphLink(source=Series[0], target=Series[1]))

In [10]:
c = (
        Graph()
        .add("", nodes, links, repulsion=100)
        .set_global_opts(title_opts=opts.TitleOpts(title="施引-被引作者关系图"))
    )
c.render_notebook()

In [97]:
file_path_2 = r'C:\Users\yyrhw\Desktop\citation_datasets\citation_author_2.csv'

In [98]:
datasource_for_name = pd.read_csv(file_path_2).drop_duplicates(subset=['被引作者'], keep='first')
datasource_for_name['施引次数'] = datasource_for_name['施引次数'] / 10 # 控制点的大小

In [99]:
datasource_for_link = pd.read_csv(file_path)

In [100]:
nodes = []
links = []
categories = []
# 注意name不能重复，否则输出图片就会变成空白
for index, Series in datasource_for_name.iterrows():
    nodes.append({'name': Series[0], 'symbolSize': Series[3], 'draggable': 'False', 
                  'value':Series[3], 'category': Series[0], 'label': {'normal': {'show': 'True'}}})
    categories.append({'name': Series[0]})
for index, Series in datasource_for_link.iterrows():
    links.append({'source': Series[0], 'target': Series[1]})

In [101]:
network_graph = (
        Graph()
        .add(
            "",
            nodes,
            links,
            categories,
            repulsion=100,
            linestyle_opts=opts.LineStyleOpts(curve=0.2),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(
            legend_opts=opts.LegendOpts(is_show=False),
            title_opts=opts.TitleOpts(title="引文网络图"),
        )
    )

In [102]:
network_graph.render_notebook()

In [104]:
network_graph.render('network_visualization.html')

'C:\\Users\\yyrhw\\summer_vacation\\network_visualization.html'